In [1]:
import LinearAlgebra as la
using BenchmarkTools: @btime
import SplitApplyCombine

In [9]:
struct HO_Funcs
    ω::Float64
    hermites::Vector{Float64}
    
    function HO_Funcs(l, ω)
        hermites = zeros(l)
        hermites[1] = 1.0
        
        return new(ω, hermites)
    end
end

In [10]:
function fast_ho(x, ho)
    (; ω, hermites) = ho
    hos = zero(hermites)
    
    x = √ω * x

    hermites[1] = 1.0
    hermites[2] = 2x

    ho_fac = (ω / π)^0.25 * exp(-x^2 / 2)
    hos[1] = ho_fac * hermites[1]
    ho_fac *= 1 / √2
    hos[2] = ho_fac * hermites[2]

    for n in 3:length(hos)
        hermites[n] = 2x * hermites[n-1] - 2(n - 2) * hermites[n-2]

        ho_fac *= 1 / sqrt( 2(n - 1) )
        hos[n] = ho_fac * hermites[n]
    end
    
    return hos
end

fast_ho (generic function with 1 method)

In [94]:
function fast_ho!(hos, x, ho)
    (; ω, hermites) = ho
    
    x = √ω * x

    hermites[1] = 1.0
    hermites[2] = 2x

    ho_fac = (ω / π)^0.25 * exp(-x^2 / 2)
    hos[1] = ho_fac * hermites[1]
    ho_fac *= 1 / √2
    hos[2] = ho_fac * hermites[2]

    for n in 3:length(hos)
        @inbounds hermites[n] = 2x * hermites[n-1] - 2(n - 2) * hermites[n-2]

        ho_fac *= 1 / sqrt( 2(n - 1) )
        @inbounds hos[n] = ho_fac * hermites[n]
    end
    
    return hos
end

fast_ho! (generic function with 1 method)

In [95]:
function spatial(grid, ho)
    n = length(grid)
    l = length(ho.hermites)
    hos = zeros(l)
    res = [zeros(n) for i in 1:l]
    
    for i in 1:n
        fast_ho!(hos, grid[i], ho)
        for j in 1:l
            @inbounds res[j][i] = hos[j]
        end
    end
    
    return res
end

spatial (generic function with 1 method)

In [96]:
@btime spatial(grid, ho);

  1.374 ms (102 allocations: 1.55 MiB)


In [13]:
grid = [x for x in range(-10, stop = 10, length = 2001)];

l_0 = 100  # number of basis functions
ω = 0.25  # strength of harmonic oscillator potential
ho = HO_Funcs(l_0, ω);

In [ ]:
function potential(x, ho::HOFunc)
    return 0.5 * ho.ω^2 * x^2
end

function E_n(ho::HOFunc)
    return (ho.order + 0.5) * ho.ω
end

# --------------------------------- The basis functions collected --------------------------------

abstract type Basis end

struct HOBasis <: Basis
    funcs::Vector{HOFunc}
    a::Float64
end

function HOBasis(l, ω, a)
    funcs = [HOFunc(i, ω) for i in 0:l-1]
    return HOBasis(funcs, a)
end

# --------------------------------- The system ----------------------------------



struct System{T<:Basis}
    n::Int64 # number of particles
    l::Int64   # number of basis functions
    
    h::Array{Float64, 2} # one-body integral
    u::Array{Float64, 4} # two-body integral
    spfs::Vector{Vector{Float64}} # the basis functions evaluated on the grid
    
    basis::T
    grid::Vector{Float64}
    
    function System(n, basis, grid)
        # The basis functions evaluated on the grid
        spfs = [compute.(grid, (ψ_i,)) for ψ_i in basis.funcs]
        l = length(basis.funcs)
        
        # One body integrals
        h = la.Diagonal([E_n(ψ_i) for ψ_i in basis.funcs])
        
        # Two body integrals
        inner = inner_ints(spfs, grid, basis.a)
        u = outer_int(spfs, grid, inner)
        
        # Adding spin
        l = l * 2
        h = kron(h, [1 0; 0 1])
        u = add_spin_u(u)
        spfs = [spfs[(i + 1)÷2] for i in 1:l]
        
        # Anti-symmetrizing u
        u = u .- permutedims(u, [1, 2, 4, 3])
        
        return new{typeof(basis)}(n, l, h, u, spfs, basis, grid)
    end
end

function trapz(f_vals, grid)::Float64 # TODO simpson!!
    val = sum(f_vals)
    val = val - 0.5 * (f_vals[1] + f_vals[end])

    return val * (grid[2] - grid[1])
end

function inner_ints(spfs::Vector{Vector{T}}, grid, a) where T<:Real
    l = length(spfs)
    inner_int = zeros(l, l, length(spfs[1]))
    f_vals = zero(grid)
    coulomb = zero(grid)
    for (xi, x1) in enumerate(grid)
        coulomb .= 1 ./ sqrt.( (grid .- x1).^2 .+ a.^2 )
        for κ in 1:l
            for λ in κ:l
                f_vals .= spfs[κ] .* coulomb .* spfs[λ]
                res = trapz(f_vals, grid)
                inner_int[κ, λ, xi] = res
                inner_int[λ, κ, xi] = res
            end
        end
    end
    return inner_int
end


function inner_ints(spfs::Vector{Vector{T}}, grid, a) where T<:Complex
    l = length(spfs)
    inner_int = zeros(l, l, length(spfs[1]))
    f_vals = zero(grid)
    coloumb = zero(grid)
    for (xi, x1) in enumerate(grid)
        coloumb .= (1 ./ sqrt.( (grid .- x1).^2 .+ a.^2))
        for κ in 1:l
            for λ in κ:l
                f_vals .= conj.(spfs[κ]) .* coloumb .* spfs[λ]
                inner_int[κ, λ, xi] = trapz(f_vals, grid)
            end
        end
    end
    return inner_int
end


function outer_int(spfs, grid, inner_ints)
    l = length(spfs)
    outer_int = zeros(l, l, l, l)
    f_vals = zero(grid)
    
    for κ in 1:l
        for λ in 1:l
            inner = inner_ints[κ, λ, :]
            for μ in 1:l
                for ν in 1:l
                    f_vals .= conj.(spfs[μ]) .* inner .* spfs[ν]
                    outer_int[μ, κ, ν, λ] = trapz(f_vals, grid)
                end
            end
        end
    end
    return outer_int
end

function add_spin_u(u_old)
    """
    When we make a spin-up and spin-down duplicate of each basis function, we get many new integrals to compute
    between these new basis functions. However, most of these integrals will be zero due to opposite spins between
    the basis functions, or they will be the same as the old integrals, if the spins align.
    
    We loop over the latter two indices in our two-body integral, ν and λ. The matrix of elements at the indeces
    u_new[:, :, ν, λ] then correspond to integrals where the first two basis functions in the integral have spins
    alternating up and down.
    
    'up-up'   'up-down'   up-up   ...
    'down-up' 'down-down' down-up ...
     up-up     up-down    up-up   ...
     ...       ...        ...     ...
    
    This 2x2 pattern of spins (that we have marked with '') repeats, and all elements in the 2x2 pattern has the same
    set of spatial functions. But only one element will be non-zero, the one that has the same spins as the basis
    functions numbered ν and λ.
    
    The code below finds which one of these elements in the 2x2 pattern will be non-zero, and then uses this to turn
    the two-body-integrals without spin into the two-body-integrals with spin.
    """
    l = size(u_old)[1] * 2
    u_new = zeros((l, l, l, l))
    
    for ν in 1:l
        for λ in 1:l
            if (ν % 2 == 1)     # --- UP ---
                if (λ % 2 == 1) # UP - UP
                    pattern = [1 0; 0 0]
                else            # UP - DOWN
                    pattern = [0 1; 0 0]
                end
            else                # --- DOWN ---
                if (λ % 2 == 1) # DOWN - UP
                    pattern = [0 0; 1 0]
                else            # DOWN - DOWN
                    pattern = [0 0; 0 1]
                end
            end
            
            ν_old = Int( ceil(ν / 2) )
            λ_old = Int( ceil(λ / 2) )
            u_new[:, :, ν, λ] .= kron(u_old[:, :, ν_old, λ_old], pattern)
        end
    end
    return u_new
end

In [7]:
l_0 = 10  # number of basis functions
ω = 0.25  # strength of harmonic oscillator potential
a = 0.25  # shielding term in coloumb interaction
grid = [x for x in range(-10, stop = 10, length = 2001)]
n = 2     # number of particles

#@time system = System(n, HOBasis(l_0, ω, a), grid);

In [ ]:
system.spfs